In [1]:
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score 
import fastapi
from typing import Optional

In [18]:


_x = [1]
_y = [1]
_prediction = ["True"]
_result = ["True"] 
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.array([_x,_y]).reshape(1, -1), _result)


DummyClassifier(constant='True', strategy='constant')

In [3]:
def model_evaluation(y_test, y_pred) :
    print('Trained Model Test Data Accuracy Score :',accuracy_score(y_test, y_pred)*100)
    testacc=accuracy_score(y_test, y_pred)
    testrecall=recall_score(y_test, y_pred)
    testprecision=precision_score(y_test, y_pred)
    print(' ')
    print(classification_report(y_test, y_pred))
    return testacc, testrecall, testprecision

In [4]:
def create_eval_df(x, y, result) :
    data = {"x": x, "y": y, "result": result}
    df = pd.DataFrame(data)
    eval_df = df.copy()
    eval_df["predict"] = model.predict(data[x,y])
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]
    eval_df["correctandtrue"] = eval_df["predict"].isin([True]) & eval_df["result"].isin([True])
    return eval_df

In [15]:
app = fastapi.FastAPI()


@app.get("/predict/")
def predict(id:int = 0 , x:float =0, y:float=0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x,y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return dict([("prediction", prediction[0])])

@app.get("/result/")
def result(id:int, result:str):
    print(result)
    _result.insert(id, result)
    return dict([("message", "Copied result for {}".format(id))])

@app.get("/retrain/")
def retrain(last_n:int):
    print ("RETRAINING over last {}".format(last_n))
    #print(_x)
    #print(_y)
    X = np.rec.fromarrays([_x, _y])[:(-1*last_n)]
    Y = np.rec.fromarrays([_result])[:(-1*last_n)]

    #print(X, Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40)

    print(X_train) 
    print(y_train)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {"message": "Model retrained", "accuracy": testacc, "recall": testrecall, "precision": testprecision}

@app.get("/new_model/")
def new_model(model:str, last_n:int, strategy:Optional[str] = None, k:Optional[int] = 0):
    print(model, last_n, strategy, k)
    if model == "Dummy" :
        model = DummyClassifier(strategy=strategy)
    elif model == "KNN" :
        model = KNeighborsClassifier(k=k)
    #model.set_params(model_params)

    message = retrain(last_n=last_n)

    returns = {"message": "Model trained {}".format(model), "train": message}
    print(returns)

    return returns

#@app.get("/evaluate_model/")
#def evaluate_model(last_n:int):
#    return {"message": "Item created", "last_n": last_n}

In [16]:
#print(predict(1, 1, 1))
#result(1, True)
new_model("Dummy", 50, "stratified")

Dummy 50 stratified 0
RETRAINING over last 50
[( 1.        ,  1.        ) ( 0.06190446, -0.56259339)
 ( 0.01067388,  0.58115774) (-0.6795449 , -0.2914611 )
 ( 0.4374637 ,  0.14976167) ( 0.38914716,  0.73173988)
 (-0.2069331 , -0.50474533) ( 0.77428605, -0.60417656)
 (-0.68588042,  0.90180499) (-0.92946333, -0.40156216)
 (-0.68678717,  0.22982302) (-0.79168145, -0.8517087 )
 (-0.23396121, -0.52141773) (-0.80060708, -0.87274869)
 (-0.0667861 ,  0.72068431) (-0.26724968,  0.2067165 )
 (-0.36985245, -0.41025476) (-0.12506836, -0.54002293)
 ( 0.85375657,  0.39852632) (-0.05286105,  0.35794812)
 (-0.83363894,  0.67953852) ( 0.85547728,  0.54619748)
 ( 0.24893982,  0.02713487) (-0.17387383, -0.59162542)
 (-0.98251237,  0.65664241) ( 0.36646562,  0.2605966 )
 ( 0.14675087, -0.18789242) ( 0.32934675,  0.90343958)
 ( 0.32300085,  0.39287823) ( 0.52302089,  0.6788165 )]
[('True',) ('False',) ('False',) ('True',) ('True',) ('True',) ('True',)
 ('False',) ('False',) ('True',) ('False',) ('True',) (

TypeError: Cannot compare structured or void to non-void arrays.

In [7]:
@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

In [ ]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())

INFO:     Started server process [14544]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


301 0.8554772842856202 0.5461974762795969
True
INFO:     127.0.0.1:57385 - "GET /predict/?id=301&x=0.8554772842856202&y=0.5461974762795969 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:57388 - "GET /result/?id=301&result=True HTTP/1.1" 200 OK
302 -0.9696114223153602 0.07406361049619048
True
INFO:     127.0.0.1:57391 - "GET /predict/?id=302&x=-0.9696114223153602&y=0.07406361049619048 HTTP/1.1" 200 OK
False
INFO:     127.0.0.1:57394 - "GET /result/?id=302&result=False HTTP/1.1" 200 OK
303 -0.773980931326403 0.46095514260714854
True
INFO:     127.0.0.1:57397 - "GET /predict/?id=303&x=-0.773980931326403&y=0.46095514260714854 HTTP/1.1" 200 OK
False
INFO:     127.0.0.1:57400 - "GET /result/?id=303&result=False HTTP/1.1" 200 OK
304 0.3891471649890308 0.7317398753131836
True
INFO:     127.0.0.1:57403 - "GET /predict/?id=304&x=0.3891471649890308&y=0.7317398753131836 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:57406 - "GET /result/?id=304&result=True HTTP/1.1" 200 OK
305 0.5276596646991629 0.925362807

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\mypet\AppData\Local\Programs\Python\Python313\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        self.scope, self.receive, self.send
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\mypet\AppData\Local\Programs\Python\Python313\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mypet\AppData\Local\Programs\Python\Python313\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\mypet\AppData\Local\Programs\Python\Python313\Lib\site-packages\starlette\applications.py", line 112, in __call__
    await sel

351 -0.6170358739941488 -0.9980894389699524
True
INFO:     127.0.0.1:57780 - "GET /predict/?id=351&x=-0.6170358739941488&y=-0.9980894389699524 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:57783 - "GET /result/?id=351&result=True HTTP/1.1" 200 OK
352 -0.28204658441705543 -0.35591458162006306
True
INFO:     127.0.0.1:57786 - "GET /predict/?id=352&x=-0.28204658441705543&y=-0.35591458162006306 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:57789 - "GET /result/?id=352&result=True HTTP/1.1" 200 OK
353 0.9365588525752473 0.8129630196625763
True
INFO:     127.0.0.1:57792 - "GET /predict/?id=353&x=0.9365588525752473&y=0.8129630196625763 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:57795 - "GET /result/?id=353&result=True HTTP/1.1" 200 OK
354 0.371707223229383 -0.3852114797535289
True
INFO:     127.0.0.1:57798 - "GET /predict/?id=354&x=0.371707223229383&y=-0.3852114797535289 HTTP/1.1" 200 OK
False
INFO:     127.0.0.1:57801 - "GET /result/?id=354&result=False HTTP/1.1" 200 OK
355 -0.027340518787944523 -0.7

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\mypet\AppData\Local\Programs\Python\Python313\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        self.scope, self.receive, self.send
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\mypet\AppData\Local\Programs\Python\Python313\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mypet\AppData\Local\Programs\Python\Python313\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\mypet\AppData\Local\Programs\Python\Python313\Lib\site-packages\starlette\applications.py", line 112, in __call__
    await sel